# Initialise dim-50 standard Gaussian dist

In [ ]:
import numpy as np
import math

In [ ]:
dim = 50

Stand_Gaus_50 = Gaussian(np.zeros(dim), np.identity(dim))

# Initialise starting points

Here is how I initialise starting positions:

In [ ]:
partial = np.identity(dim)
starting_pts = np.concatenate((np.array([[0]*dim]), 1*partial, 2*partial, 3*partial, 
                               (-1)*partial, (-2)*partial, (-3)*partial), axis=0)

another_half = np.concatenate((partial[1:], np.array([[1] + [0]*(dim-1)])), axis=0) + partial

starting_pts = np.concatenate((starting_pts, another_half, 2*another_half, 3*another_half, 
                               (-1)*another_half, (-2)*another_half, (-3)*another_half), axis=0)


# Run BPS algorithm on Gaussian. Save the data.

In [ ]:
n_batch = 0
time_trajectory = 100000
lambda_ref = 1
prob_dist = Stand_Gaus_50


for i in range(20):
  start_x = starting_pts[100*n_batch + i]
  if np.linalg.norm(start_x) == 0:
    start_v = np.array([1] + [0]*(dim-1))
  else:
    start_v = start_x
  
  turn_pts, list_of_velo, striding_times, total_evals_list, computational_times\
  = BPS_basic(x0 = start_x, v0 = start_v, Time = time_trajectory, lambda_ref = lambda_ref, 
              prob_dist = prob_dist)


  turning_points = np.array(turn_pts)
  v_list = np.array(list_of_velo)
  stride_times = np.array(striding_times)
  evaluations_list = np.array(total_evals_list)
  CPUtime_list = np.array(computational_times)



  path_to_save_chain = 'BPS_data/BPS_chain_' + str(100*n_batch + i) + '_'
  np.savez(path_to_save_chain + 'turning_points', turning_points = turning_points)
  np.savez(path_to_save_chain + 'v_list', v_list = v_list)
  np.savez(path_to_save_chain + 'stride_times', stride_times = stride_times)
  np.savez(path_to_save_chain + 'evaluations_list', evaluations_list = evaluations_list)
  np.savez(path_to_save_chain + 'CPUtime_list', CPUtime_list = CPUtime_list)

  
  print('finishes the {}th run of BPS chain    cpu_time: {} sec'.format(100*n_batch + i, 
                                                                        computational_times[-1]))


print('finishes executing a whole batch of BPS chains!')


Process data for plotting KL divergence.

In [ ]:
# plot_step_size means that I would like to extract the velocity and position 
# at time=100 of the trajectory.

plot_step_size = 100
n_batch = 0
num_files = 20
starting_file = 0
all_chains = []

start_time = perf_counter()

for i in range(num_files):
  
  path_to_load_chain = 'BPS_data/BPS_chain_' + str(100*n_batch + starting_file + i) + '_'
  x_load = np.load(path_to_load_chain + 'turning_points.npz')
  v_load = np.load(path_to_load_chain + 'v_list.npz')
  t_load = np.load(path_to_load_chain + 'stride_times.npz')

  x = x_load['turning_points']
  v = v_load['v_list']
  t = t_load['stride_times']

  interm_times = np.arange(0,100000.1,plot_step_size)

  x_100s, v_100s, t_100s = x_v_t_arbitrary_times(turn_pts=x, list_of_velo=v, striding_times=t, intermediate_times=interm_times)


  path_to_save_chain = 'processed_BPS_data/processed_BPS_chain_' + str(100*n_batch + starting_file + i) + '_'
  np.savez(path_to_save_chain + 'x_array', x_array = np.array(x_100s))
  np.savez(path_to_save_chain + 'v_array', v_array = np.array(v_100s))

  print('finishes_processing {}th chain    cpu_time: {} sec'.format(100*n_batch + starting_file + i, perf_counter() - start_time))


print('finishes processing all!')